# Deep Convolutional GAN 
Reference : https://arxiv.org/abs/1511.06434

### Target 
DC GAN on celebA, cifar-10, cifar-100 datasets
###### To report
+ Train - parameters and architecture 
+ saved weights for 3 datasets 
+ loss, number of epochs
+ generated images : modes vs quality

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


#### plan

+ read to tf.Dataset() format
+ Intialize with parameters
+ model class ()
+ `noise(batch_size, dim, distribution='uniform'):`  returns `tf.data.Dataset()`
+ def `generator(z):` returns image
+ def `discriminator(img):` returns probability
+ def `batch_norm` : returns normalized batch
+ def `**LOSS**(real, fake):` returns losses of G and D
+ def `save():` saves model to a file (probably `.h5` file)
+ def `load(filename):` loads weights of the model
+ def `generate_images(noise)`
+ def `evaluate(x_test)`

#### Architecture guidelines for stable Deep Convolutional GANs - reference: GAN paper
+ Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
+ Use batchnorm in both the generator and the discriminator. _Do not apply batchnorm to G output layer and D input layer_
+ Remove fully connected hidden layers for deeper architectures.
+ Use ReLU activation in generator for all layers except for the output, which uses Tanh.
+ Use LeakyReLU activation in the discriminator for all layers. In the LeakyReLU, the slope of the leak was set to 0.2 in all models.
+ Scale images to range of $[-1,1]$
+ minibatch stocastic gradient - batch_size = 128
+ All weights were initialized from a zero-centered Normal distribution with standard deviation 0.02.

+ Adam optimizer: learning_rate = 0.0002, β1 = 0.5
+ We found global average pooling increased model stability but hurt convergence speed. A middle ground of directly connecting the highest convolutional features to the input and output respectively of the generator and discriminator worked well.
+ simple image de-duplication process. We fit a 3072-128-3072 de-noising dropout regularized RELU autoencoder on 32x32 downsampled center-crops of training examples.
+ 

In [9]:
class DC_GAN():
    def __init__(self, sess, real_img_dim, gen_img_dim, noise_dim, batch_size, num_channels=3):
        self.sess = sess
        self.noise_dim = noise_dim
        self.D_input_dim = real_img_dim
        self.G_output_dim = gen_img_dim
        self.batch_size = batch_size
        self.channels = num_channels
        self.saver = tf.train.Saver()
        
    def noise(self, batch_size, noise_dim, pdf='uniform'): 
        '''
        Inputs: batch size while training, dimension of noise
        returns random sample from the distribution - pdf (uniform by default)
        '''
        batch_size = self.batch_size
        noise_dim = self.noise_dim
        if pdf=='uniform':
            return tf.random_uniform(minval=-1, maxval=1, shape=(batch_size, noise_dim), name='z')
        if pdf=='normal':
            return tf.random_normal(mean=0, stddev=1, shape=(batch_size, noise_dim), name='z')
        
    def batch_norm(self, batch):
        return tf.contrib.layers.batch_norm(inputs=batch,
            decay=0.9,
            updates_collections=None,
            epsilon=1e-5,
            scale=True,
            is_training=True
            )
    
    def add_conv_layer(self, inputs, filter_size, in_channels, out_channels, scope_name):
        with tf.variable_scope(scope_name):
            shape = [filter_size, filter_size, in_channels, out_channels]
            weights = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
            biases = tf.Variable(tf.constant(0.05, shape=[out_channels]))
            
            layer = tf.nn.conv2d(
                input=inputs,
                filter=weights,
                strides=[1, 2, 2, 1],
                padding='SAME') + biases
            
            layer = tf.nn.leaky_relu(self.batch_norm(layer), alpha=0.2)
            return layer, weights, biases
            
    def add_conv_transpose_layer(self,inputs,filter_size,in_channels,out_channels,scope_name,activation):
        with tf.variable_scope(scope_name):
            shape = [filter_size, filter_size, out_channels, in_channels]
            weights = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
            biases = tf.Variable(tf.constant(0.05, shape=[out_channels]))

            layer = tf.nn.conv2d_transpose(
                input=inputs,
                filter=weights,
                strides=[1, 2, 2, 1],
                padding='SAME') + biases

            if activation == 'relu': layer = tf.nn.relu(self.batch_norm(layer))
            elif activation == 'tanh': layer = tf.nn.tanh(self.batch_norm(layer))

            return layer, weights, biases

    def discriminator(self, img):
        scope_name='discriminator'
        h1, w1, b1 = self.add_conv_layer(
            inputs = img,
            filter_size=5,
            in_channels=self.num_channels,
            out_channels=64,
            scope_name=scope_name,
        )
        h2, w2, b2 = self.add_conv_layer(
            inputs=h1,
            filter_size=5,
            in_channels=64,
            num_filters=128,
            scope_name=scope_name,
        )
        h3, w3, b3 = self.add_conv_layer(
            inputs=h2,
            filter_size=5,
            in_channels=128,
            out_channels=256,
            scope_name=scope_name,
        )
        h4, w4, b4 = self.add_conv_layer(
            inputs=h3,
            filter_size=5,
            in_channels=256,
            out_channels=512,
            scope_name=scope_name,
        )
        logits = tf.layers.dense(
            inputs=tf.reshape(h4, shape=[self.batch_size, -1]),
            units=1,
            use_bias=True,
            kernel_initalizer=tf.contrib.layers.xavier_initializer()
        )
        return logits
        
    def generator(self, z):
        scope_name = 'generator'
        _z = tf.layers.dense(
            inputs=z,
            units=4*4*1024,
            use_bias=True,
            kernel_initalizer=tf.contrib.layers.xavier_initializer()
        ) # projection of z to 4x4x1024 layer (linear)
        h1 = tf.reshape(_z, [-1, 4,4,1024])
        h2, w2, b2 = self.add_conv_transpose_layer(
            inputs=h1,
            filter_size=5,
            in_channels=1024,
            out_channels=512,
            scope_name=scope_name,
            activation='relu'
        )
        h3, w3, b3 = self.add_conv_transpose_layer(
            inputs=h2,
            filter_size=5,
            in_channels=512,
            out_channels=256,
            scope_name=scope_name,
            activation='relu'
        )
        h4, w4, b4 = self.add_conv_transpose_layer(
            inputs=h4,
            filter_size=5,
            in_channels=256,
            out_channels=128,
            scope_name=scope_name,
            activation='relu'
        )
        G_out, w5, b5 = self.add_conv_transpose_layer(
            inputs=h4,
            filter_size=5,
            in_channels=128,
            num_filters=self.num_channels,
            scope_name=scope_name,
            activation='tanh'
        )
        return G_out
    